This file is used to generate cycle_dataset. It outputs a file named cycle_dataset.npz.

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

TRAIN_NUM = 60000
TEST_NUM = 10000

data = None
elfs = None

U = (0,-1)
D = (0,1)
L = (-1, 0)
R = (1, 0)

four_directions = [(0,1),(1,0),(0,-1),(-1,0)]
five_directions = [(0,1),(1,0),(0,-1),(-1,0),(0,0)]


O = (30,30)

def make_random_direction():
    random_direction_x = 0
    random_direction_y = random.randint(0,1) * 2 - 1
    if random.randint(0,1):
        random_direction_x, random_direction_y = random_direction_y, random_direction_x
    return random_direction_x, random_direction_y

def turn_direction(dir):
    x,y = dir
    if random.randint(0,1):
        return y,x
    return -y,-x


def color(place):
    data[place[0],place[1]] = 1

def iscolored(place):
    return data[place[0],place[1]] == 1


def addp(p1,p2):
    return p1[0] + p2[0], p1[1] + p2[1]

class HasCycle(Exception):
    pass


class elf:
    def __init__(self,place,dir):
        self.place = place
        self.dir = dir
    def move(self):
        self.place = addp(self.place, self.dir)
        color(self.place)
        nextmove = addp(self.place, self.dir)
        for direction in five_directions:
            if addp(self.dir, direction) == (0, 0):
                continue
            if iscolored(addp(nextmove, direction)):
                color(nextmove)
                raise HasCycle
    def turn(self):
        self.dir = turn_direction(self.dir)
        self.move()
        self.move()
    def split(self):
        anotherself = deepcopy(self)
        anotherself.turn()
        elfs.append(anotherself)
        self.move()
        self.move()

def init():
    global elfs
    global data
    data = np.zeros((60,60),dtype = int)
    data[O[0],O[1]] = 1
    elfs = []
    elfs.append(elf(O,make_random_direction()))

def runelfs():
    moving_elf = random.choice(elfs)
    r = random.random()
    if(r < 0.8):
        moving_elf.move()
    elif(r < 0.9):
        moving_elf.turn()
    else:
        moving_elf.split()

cycle_data = [[] for _ in range(100)]
no_cycle_data = [[] for _ in range(100)]

def lessthan26(v):
    i = 0
    j = len(v) - 1
    while not v[i]:
        i += 1
    while not v[j]:
        j -= 1
    return j - i + 1 <= 26

def center(v):
    i = 0
    j = len(v) - 1
    while not v[i]:
        i += 1
    while not v[j]:
        j -= 1
    return (i + j) // 2 + 1

def valid(d):
    return lessthan26(np.array(d).any(axis=0)) and lessthan26(np.array(d).any(axis=1))

def centralize(d):
    cx = center(np.array(d).any(axis=1))
    cy = center(np.array(d).any(axis=0))
    return d[cx-14:cx+14,cy-14:cy+14].copy()

def make_data():
    init()
    try:
        for i in range(random.randint(40,60)):
            runelfs()
    except HasCycle:
        if valid(data):
            cycle_data[i].append(centralize(data))
        return
    except IndexError:
        return
    if valid(data):
        no_cycle_data[i].append(centralize(data))
    return




for _ in range(10*(TRAIN_NUM+TEST_NUM)):
    make_data()
cy = []
for i in range (40,48):
    cy += cycle_data[i]
cy = random.sample(cy,(TRAIN_NUM+TEST_NUM)//2)
ncy = []
for i in range (40,48):
    ncy += no_cycle_data[i]
ncy = random.sample(ncy,(TRAIN_NUM+TEST_NUM+1)//2)
ltmp = [(i,1) for i in cy[:TRAIN_NUM//2]] + [(i,0) for i in ncy[:TRAIN_NUM//2]]
random.shuffle(ltmp)
train_images, train_labels = zip(*ltmp)
train_images = np.array(train_images)
train_labels = np.array(train_labels)

ltmp = [(i,1) for i in cy[TRAIN_NUM//2:]] + [(i,0) for i in ncy[TRAIN_NUM//2:]]
test_images, test_labels = zip(*ltmp)
test_images = np.array(test_images)
test_labels = np.array(test_labels)
np.savez_compressed('cycle_dataset.npz', train_images = train_images, train_labels = train_labels, test_images = test_images, test_labels = test_labels)

How to load and plot:

```python
import numpy as np
import matplotlib.pyplot as plt

cycle_dataset = np.load('cycle_dataset.npz')
train_images = cycle_dataset['train_images']
train_labels = cycle_dataset['train_labels']
test_images = cycle_dataset['test_images']
test_labels = cycle_dataset['test_labels']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()
def show(X):
    '''
    Changing inf to 40 can get a picture with more detail.
    '''
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            if X[i,j] > 1000:
                X[i,j] = 40
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X)
    plt.show()
```